In [1]:
import rebound
import pandas as pd
from rebound import InterruptiblePool
import numpy as np
import os
import time

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import time 
from scipy import stats
import rebound
from scipy.special import gamma
from scipy.stats import norm, ks_2samp
from celmech.disturbing_function import *
from sympy import symbols, init_printing,Function,simplify,collect,cos
init_printing()
import numpy as np
from celmech.poincare import Poincare, PoincareHamiltonian
from celmech import Andoyer, AndoyerHamiltonian
from celmech.resonances import resonant_period_ratios, resonance_intersections_list, resonance_pratio_span
from rebound.plotting import OrbitPlotOneSlice
import itertools
import time
path = '/mnt/raid-cita/dtamayo/newstabilitydataset/data/resonant/simulation_archives/runs/'

ImportError: No module named celmech.disturbing_function

In [ ]:
def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

In [ ]:
def worker(df):
    Ndump=10
    for i in range(df.shape[0]):
        #try:
        run(df, i, tmax=1.e4)
        #except:
        #    for col in columns:
        #        df.loc[df.index[i], col] = np.nan
        if i%Ndump == 0:
            df.to_csv('restempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')
    df.to_csv('restempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')

In [ ]:
def findres(sim, i1, i2):
    delta = 0.03
    maxorder = 2
    ps = Poincare.from_Simulation(sim=sim).particles # get averaged mean motions
    n1 = ps[i1].n
    n2 = ps[i2].n
    
    m1 = ps[i1].m
    m2 = ps[i2].m
    
    Pratio = n2/n1
    res = resonant_period_ratios(Pratio-delta,Pratio+delta, order=maxorder)
    
    Z = np.sqrt((ps[i1].e*np.cos(ps[i1].pomega) - ps[i2].e*np.cos(ps[i2].pomega))**2 + (ps[i1].e*np.sin(ps[i1].pomega) - ps[i2].e*np.sin(ps[i2].pomega))**2)
    
    maxstrength = 0
    j, k, i1, i2, strength = -1, -1, -1, -1, -1
    for a, b in res:
        s = np.abs(np.sqrt(m1+m2)*Z**((b-a)/2.)/(b*n2 - a*n1))
        #print('{0}:{1}'.format(b, a), (b*n2 - a*n1), s)
        if s > maxstrength:
            j = b
            k = b-a
            i1 = 1
            i2 = 2
            strength=s
            maxstrength = s
            
    return j, k, strength

In [ ]:
def run(df, index, tmax=1.e4, runcel=False):
    t0 = time.time()
    file = path+"sa{0}".format(df.loc[df.index[index], 'runstring'])
    sim = rebound.Simulation.from_file(file)
    ps = sim.particles
    sim.init_megno()
    
    N = sim.N - sim.N_var
    a0 = [0] + [sim.particles[i].a for i in range(1, N)]
    Npairs = int((N-1)*(N-2)/2)
    js, ks, strengths = np.zeros(Npairs, dtype=np.int), np.zeros(Npairs, dtype=np.int), np.zeros(Npairs)
    maxj, maxk, maxi1, maxi2, maxpairindex, maxstrength = -1, -1, -1, -1, -1, -1

    Zcross = np.zeros(Npairs)
    #print('pairindex, i1, i2, j, k, strength')
    for i, [i1, i2] in enumerate(itertools.combinations(np.arange(1, N), 2)):
        js[i], ks[i], strengths[i] = findres(sim, i1, i2)
        Zcross[i] = (ps[int(i2)].a-ps[int(i1)].a)/ps[int(i1)].a
        #print(i, i1, i2, js[i], ks[i], strengths[i])
        if strengths[i] > maxstrength:
            maxj, maxk, maxi1, maxi2, maxpairindex, maxstrength = js[i], ks[i], i1, i2, i, strengths[i]
    
    df.loc[df.index[index], 'Zcross12'] = Zcross[0]
    df.loc[df.index[index], 'Zcross13'] = Zcross[1]
    df.loc[df.index[index], 'Zcross23'] = Zcross[2]
    df.loc[df.index[index], 'maxj'] = maxj
    df.loc[df.index[index], 'maxk'] = maxk
    df.loc[df.index[index], 'maxi1'] = maxi1
    df.loc[df.index[index], 'maxi2'] = maxi2
    df.loc[df.index[index], 'maxstrength'] = maxstrength
    
    sortedstrengths = strengths.copy()
    sortedstrengths.sort()
    if sortedstrengths[-1] > 0 and sortedstrengths[-2] > 0:
        df.loc[df.index[index], 'secondres'] = sortedstrengths[-2]/sortedstrengths[-1]
    else:
        df.loc[df.index[index], 'secondres'] = -1
        
    if maxstrength > 0 and runcel is True:
        #print(maxj, maxk, maxi1, maxi2, maxstrength)
        andvars = Andoyer.from_Simulation(sim=sim, j=maxj, k=maxk, a10=a0[maxi1], i1=maxi1, i2=maxi2)
        H = AndoyerHamiltonian(andvars)
        #print(maxj, maxk, maxi1, maxi2, H.state.phi)
    else:
        H = None

    #print('max', maxi1, maxi2, maxj, maxk, maxpairindex, maxstrength)
    #print('df (j, k, pairindex):', df.loc[df.index[index], 'j'], df.loc[df.index[index], 'k'], df.loc[df.index[index], 'pairindex'])

    Nout = 1000
    times = np.linspace(0, tmax, Nout)

    eminus = np.zeros((Npairs, Nout))
    rebound_Z, rebound_phi = np.zeros((Npairs,Nout)), np.zeros((Npairs,Nout))
    rebound_Zcom, rebound_phiZcom = np.zeros((Npairs,Nout)), np.zeros((Npairs,Nout))
    rebound_Zstar, rebound_dKprime = np.zeros((Npairs,Nout)), np.zeros((Npairs,Nout))
    celmech_Z, celmech_phi = np.zeros(Nout), np.zeros(Nout)
    celmech_Zcom, celmech_phiZcom = np.zeros(Nout), np.zeros(Nout)
    celmech_Zstar, celmech_dKprime = np.zeros(Nout), np.zeros(Nout)

    for i,t in enumerate(times):
        for j, [i1, i2] in enumerate(itertools.combinations(np.arange(1, N), 2)):
            i1, i2 = int(i1), int(i2)
            eminus[j, i] = np.sqrt((ps[i2].e*np.cos(ps[i2].pomega)-ps[i1].e*np.cos(ps[i1].pomega))**2 + (ps[i2].e*np.sin(ps[i2].pomega)-ps[i1].e*np.sin(ps[i1].pomega))**2)
            if js[j] != -1:
                avars = Andoyer.from_Simulation(sim=sim, j=js[j], k=ks[j], a10=a0[i1], i1=i1, i2=i2)
                rebound_Z[j, i] = avars.Z
                rebound_phi[j, i] = avars.phi
                rebound_Zcom[j, i] = avars.Zcom
                rebound_phiZcom[j, i] = avars.phiZcom
                rebound_Zstar[j, i] = avars.Zstar
                rebound_dKprime[j, i] = avars.dKprime
        
        if maxstrength > 0 and runcel is True:
            celmech_Z[i] = H.state.Z
            celmech_phi[i] = H.state.phi
            celmech_Zcom[i] = H.state.Zcom
            celmech_phiZcom[i] = H.state.phiZcom
            celmech_Zstar[i] = H.state.Zstar
            celmech_dKprime[i] = H.state.dKprime

            H.integrate(t)
        try:
            sim.integrate(t)
        except:
            break
        
    mask = eminus[0] > 0 # where there are data points in case sim ends early
    times = times[mask]
    eminus = eminus[:, mask]
    rebound_Z, rebound_phi = rebound_Z[:, mask], rebound_phi[:, mask]
    rebound_Zcom, rebound_phiZcom =  rebound_Zcom[:, mask], rebound_phiZcom[:, mask]
    rebound_Zstar, rebound_dKprime = rebound_Zstar[:, mask], rebound_dKprime[:, mask]
    celmech_Z, celmech_phi, celmech_Zcom, celmech_phiZcom = celmech_Z[mask], celmech_phi[mask], celmech_Zcom[mask], celmech_phiZcom[mask]
    celmech_Zstar, celmech_dKprime = celmech_Zstar[mask], celmech_dKprime[mask]
    
    for i, s in zip([0,2], ['12', '23']): # take adjacent ones
        EM = eminus[i]
        Zc = Zcross[i]
        df.loc[df.index[index], 'EMmed'+s] = np.median(EM)/Zc
        df.loc[df.index[index], 'EMmax'+s] = EM.max()/Zc
        p = np.poly1d(np.polyfit(times, EM, 3))
        m = p(times)
        EMdrift = np.abs((m[-1]-m[0])/m[0])
        df.loc[df.index[index], 'EMdrift'+s] = EMdrift
        maxindex = (m == m.max()).nonzero()[0][0] # index where cubic polynomial fit to EM reaches max to track long wavelength variations (secular?)
        if EMdrift > 0.1 and (maxindex < 0.01*Nout or maxindex > 0.99*Nout): # don't flag as not capturing secular if Z isn't varying significantly in first place
            df.loc[df.index[index], 'capseculartscale'+s] = 0
        else:
            df.loc[df.index[index], 'capseculartscale'+s] = 1
        df.loc[df.index[index], 'EMdetrendedstd'+s] = pd.Series(EM-m).std()/EM[0]
        rollstd = pd.rolling_std(EM, window=100)
        df.loc[df.index[index], 'EMrollingstd'+s] = np.median(rollstd[100:])/EM[0]
        var = [EM[:j].var() for j in range(len(EM))]
        p = np.poly1d(np.polyfit(times[len(var)//2:], var[len(var)//2:], 1)) # fit only second half to get rid of transient
        df.loc[df.index[index], 'DiffcoeffEM'+s] = p[1]/Zc**2
        df.loc[df.index[index], 'medvarEM'+s] = np.median(var[len(var)//2:])/Zc**2
        if strengths[i] != -1:
            Z = rebound_Z[i]
            df.loc[df.index[index], 'Zmed'+s] = np.median(Z)/Zc
            df.loc[df.index[index], 'Zmax'+s] = rebound_Z[i].max()/Zc
            p = np.poly1d(np.polyfit(times, Z, 3))
            m = p(times)
            df.loc[df.index[index], 'Zdetrendedstd'+s] = pd.Series(Z-m).std()/Z[0]
            rollstd = pd.rolling_std(Z, window=100)
            df.loc[df.index[index], 'Zrollingstd'+s] = np.median(rollstd[100:])/Z[0]
            var = [Z[:j].var() for j in range(len(Z))]
            p = np.poly1d(np.polyfit(times[len(var)//2:], var[len(var)//2:], 1)) # fit only second half to get rid of transient
            df.loc[df.index[index], 'DiffcoeffZ'+s] = p[1]/Zc**2
            df.loc[df.index[index], 'medvarZ'+s] = np.median(var[len(var)//2:])/Zc**2
            df.loc[df.index[index], 'Zcomdrift'+s] = np.max(np.abs(rebound_Zcom[i]-rebound_Zcom[i, 0])/rebound_Zcom[i, 0])
            rollstd = pd.rolling_std(rebound_Zcom[i], window=100)
            df.loc[df.index[index], 'Zcomrollingstd'+s] = np.median(rollstd[100:])/rebound_Zcom[i,0]
            df.loc[df.index[index], 'phiZcomdrift'+s] = np.max(np.abs(rebound_phiZcom[i]-rebound_phiZcom[i, 0]))
            rollstd = pd.rolling_std(rebound_phiZcom[i], window=100)
            df.loc[df.index[index], 'phiZcomrollingstd'+s] = np.median(rollstd[100:])
            df.loc[df.index[index], 'Zstardrift'+s] = np.max(np.abs(rebound_Zstar[i]-rebound_Zstar[i, 0])/rebound_Zstar[i, 0])
            rollstd = pd.rolling_std(rebound_Zcom[i], window=100)
            df.loc[df.index[index], 'Zstarrollingstd'+s] = np.median(rollstd[100:])/rebound_Zstar[i,0]
            Zcosphi = Z*np.cos(rebound_phi[i])
            df.loc[df.index[index], 'Zcosphistd'+s] = Zcosphi.std()/Zc
            df.loc[df.index[index], 'medZcosphi'+s] = np.median(Zcosphi)/Zc
    df.loc[df.index[index], 'tlyap'] = 1/sim.calculate_lyapunov()
    df.loc[df.index[index], 'featgentime'] = time.time()-t0
    return sim, times, maxj, maxk, maxi1, maxi2, maxpairindex, maxstrength, Npairs, js, ks, strengths, eminus, rebound_Z, rebound_phi, rebound_Zcom, rebound_phiZcom, rebound_Zstar, rebound_dKprime, celmech_Z, celmech_phi, celmech_Zcom, celmech_phiZcom, celmech_Zstar, celmech_dKprime, Zcross

In [3]:
columns = ['featgentime', 'Zcross12', 'Zcross13', 'Zcross23', 'maxj', 'maxk', 'maxi1', 'maxi2', 'maxstrength', 'secondres', 'tlyap']
for i, s in zip([0,2], ['12', '23']): # take adjacent ones
    columns += ['EMmed'+s, 'EMmax'+s, 'EMdrift'+s, 'capseculartscale'+s, 'EMdetrendedstd'+s, 'EMrollingstd'+s, 'DiffcoeffEM'+s, 'medvarEM'+s, 'Zmed'+s, 'Zmax'+s, 'Zdetrendedstd'+s, 'Zrollingstd'+s, 'DiffcoeffZ'+s, 'medvarZ'+s, 'Zcomdrift'+s, 'Zcomrollingstd'+s, 'phiZcomdrift'+s, 'phiZcomrollingstd'+s, 'Zstardrift'+s, 'Zstarrollingstd'+s, 'Zcosphistd'+s, 'medZcosphi'+s]
columns

['featgentime',
 'Zcross12',
 'Zcross13',
 'Zcross23',
 'maxj',
 'maxk',
 'maxi1',
 'maxi2',
 'maxstrength',
 'secondres',
 'tlyap',
 'EMmed12',
 'EMmax12',
 'EMdrift12',
 'capseculartscale12',
 'EMdetrendedstd12',
 'EMrollingstd12',
 'DiffcoeffEM12',
 'medvarEM12',
 'Zmed12',
 'Zmax12',
 'Zdetrendedstd12',
 'Zrollingstd12',
 'DiffcoeffZ12',
 'medvarZ12',
 'Zcomdrift12',
 'Zcomrollingstd12',
 'phiZcomdrift12',
 'phiZcomrollingstd12',
 'Zstardrift12',
 'Zstarrollingstd12',
 'Zcosphistd12',
 'medZcosphi12',
 'EMmed23',
 'EMmax23',
 'EMdrift23',
 'capseculartscale23',
 'EMdetrendedstd23',
 'EMrollingstd23',
 'DiffcoeffEM23',
 'medvarEM23',
 'Zmed23',
 'Zmax23',
 'Zdetrendedstd23',
 'Zrollingstd23',
 'DiffcoeffZ23',
 'medvarZ23',
 'Zcomdrift23',
 'Zcomrollingstd23',
 'phiZcomdrift23',
 'phiZcomrollingstd23',
 'Zstardrift23',
 'Zstarrollingstd23',
 'Zcosphistd23',
 'medZcosphi23']

In [4]:
df = pd.read_csv('/mnt/raid-cita/dtamayo/newstabilitydataset/data/csvs/data.csv', index_col=0)
df['SimID'] = np.arange(df.shape[0])
df['Stable'] = df['instability_time'] >= 9.99e8
df = df[['SimID', 'runstring']]#'Stable', 'instability_time', 'runstring', 'seed']]
for col in columns:
    df[col] = 0
df.set_index('SimID', inplace=True)

df.tail()

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
9995,0009995.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0009996.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0009997.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0009998.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999,0009999.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
featuresfile='resshortfeatures.csv'

In [23]:
df.to_csv(featuresfile, encoding='ascii')

In [52]:
Nworkers=24
df = pd.read_csv(featuresfile, index_col=0)
df.tail()

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
9995,0009995.bin,4.138066,0.160673,0.603900,0.381870,5.0,1.0,1.0,2.0,0.361500,...,2.948659e-09,0.000008,0.030713,0.004626,0.100482,0.003975,0.031589,0.003456,0.056012,0.000215
9996,0009996.bin,2.239034,0.052687,0.525926,0.449554,27.0,2.0,1.0,2.0,0.000380,...,-4.845470e-11,0.000004,0.386751,0.032375,5.882195,0.039279,0.000000,0.000000,0.006654,-0.000014
9997,0009997.bin,2.366055,0.084531,0.206673,0.112622,4.0,1.0,1.0,3.0,0.007938,...,7.329326e-07,0.006643,0.884267,0.154314,1.528973,0.217228,8.763043,1.017298,0.243900,-0.019435
9998,0009998.bin,2.827924,0.284352,0.365163,0.062919,3.0,1.0,1.0,2.0,0.000939,...,3.538395e-09,0.000067,0.169636,0.005230,0.382835,0.011517,0.034858,0.006005,0.105414,0.006507
9999,0009999.bin,2.549488,0.129620,0.389971,0.230476,6.0,1.0,1.0,2.0,0.046137,...,1.570898e-08,0.003621,3.935178,0.275869,0.756752,0.091269,1.592176,0.264800,0.112955,-0.003568


In [53]:
df = df[df['featgentime'] == 0.]
print(df.shape)
IDs = df.index.values
splits = chunkify(IDs, Nworkers)
dfs = [df.loc[split] for split in splits]

(20, 56)


In [54]:
pool = InterruptiblePool(Nworkers)
pool.map(worker, dfs)

/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)


ValueError: ('Cannot read binary file. Check filename and file contents.', 1)

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:112: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:125: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:127: RuntimeWarning: Degrees of freedom <= 0 for slice
/

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:127: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:132: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:132: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:135: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:135: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:138: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:135: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:135: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:138: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:138: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:112: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:125: FutureWarning: pd.rolling_std

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:127: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:125: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:127: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:132: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:135: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:138: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/

In [55]:
import os

df = pd.read_csv(featuresfile, index_col=0)

for f in os.listdir('restempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('restempcsvs/'+f, index_col=0)
        df.update(newdf)
        
#test = test.sort_values('SimID')
#test = test.reset_index(drop=True)
df[df['featgentime'] == 0.].shape

20781.csv
20780.csv
20775.csv
20782.csv
20766.csv
20765.csv
20786.csv
20778.csv
20770.csv
20783.csv
20785.csv
20779.csv
20776.csv
20767.csv
20769.csv
20787.csv
20773.csv
20768.csv
20771.csv
20777.csv
20774.csv
20784.csv
20772.csv


In [35]:
df.head()

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
0,0000000.bin,2.588877,0.504063,0.594658,0.060234,12.0,1.0,2.0,3.0,0.048244,...,-5.249295e-10,1.471706e-04,0.022954,0.000753,0.144779,0.004245,0.167833,0.001449,0.013174,-0.342356
1,0000001.bin,5.023860,0.240504,0.340424,0.080547,9.0,1.0,2.0,3.0,0.020943,...,1.420017e-09,9.234751e-05,0.037671,0.001800,0.050951,0.002006,0.027171,0.000309,0.069005,-0.684871
2,0000002.bin,3.386137,0.129660,1.261558,1.001981,6.0,1.0,1.0,2.0,0.067762,...,9.176673e-11,3.508728e-07,0.427300,0.013722,0.662545,0.019659,0.004258,0.001175,0.020689,0.000014
3,0000003.bin,3.726120,0.406112,1.007603,0.427768,5.0,2.0,1.0,2.0,0.045905,...,5.512634e-08,3.856095e-04,0.837207,0.054084,0.949622,0.048041,0.000000,0.000000,0.081755,0.001596
4,0000004.bin,3.290068,0.059897,0.332922,0.257596,12.0,1.0,1.0,2.0,0.030171,...,-5.059799e-09,1.147590e-03,4.493881,0.296834,5.268717,0.106801,0.000000,0.012919,0.082736,0.000399


In [57]:
df.to_csv(featuresfile, encoding='ascii')

In [56]:
df[df['featgentime'] == 0]

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
2058,0002058.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
def worker(df):
    Ndump=10
    for i, row in df.iterrows():
        run(df, row['SimID'])
        if i%Ndump == 0:
            df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')
    df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')

In [48]:
def run(df, ID):
    if ID % 2 == 0:
        df.loc[df['SimID']==ID, 'featgentime'] = 7

In [300]:
test.head()

,SimID,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
0,0,False,1.545872e+06,0000000.bin,3500000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,True,9.990000e+08,0000001.bin,700001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,True,9.990000e+08,0000002.bin,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,False,2.287671e+06,0000003.bin,2100003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,False,9.668931e+05,0000004.bin,700004,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
Nworkers=3
test = test[test['featgentime'] == 0]
print(test.shape)
IDs = test['SimID'].values
splits = chunkify(IDs, Nworkers)
dfs = [test[test['SimID'].isin(split)] for split in splits]
pool = InterruptiblePool(Nworkers)
pool.map(worker, dfs)

(33, 35)


[None, None, None]

In [84]:
import os

test = pd.DataFrame(columns=test.columns)
for f in os.listdir('tempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('tempcsvs/'+f, index_col=0)
        test = test.append(newdf)
        
test = test.sort_values('SimID')
test = test.reset_index(drop=True)
test.head()

10377.csv
10381.csv
10371.csv
10375.csv
10385.csv
10376.csv
10384.csv
10386.csv
10379.csv
10383.csv
10382.csv
10378.csv
10373.csv


,DiffcoeffZ12,DiffcoeffZ23,SimID,Stable,Zcosphistd12,Zcosphistd23,Zcross12,Zcross13,Zcross23,Zdetrendedstd12,...,maxi2,maxj,maxk,maxstrength,medvarZ12,medvarZ23,runstring,secondres,secondstrength,seed
0,0.000000e+00,-5.249295e-10,0.0,False,0.000000,0.012035,0.504063,0.594658,0.060234,0.000000,...,3.0,12.0,1.0,0.048244,0.000000,1.471706e-04,0000000.bin,0.195509,0.0,3500000.0
1,5.530677e-11,9.176673e-11,2.0,True,0.003070,0.000785,0.129660,1.261558,1.001981,0.003071,...,2.0,6.0,1.0,0.067762,0.000009,3.508728e-07,0000002.bin,0.029946,0.0,2.0
2,-1.838390e-08,-5.059799e-09,4.0,False,0.023602,0.033524,0.059897,0.332922,0.257596,0.022934,...,2.0,12.0,1.0,0.030171,0.000586,1.147590e-03,0000004.bin,0.140455,0.0,700004.0
3,2.151018e-08,-1.581556e-07,5.0,False,0.011011,0.053589,0.185250,0.259467,0.062617,0.001381,...,3.0,23.0,2.0,0.001197,0.000029,2.971640e-03,0000005.bin,0.567226,0.0,1400005.0
4,0.000000e+00,-5.992155e-10,6.0,False,0.000000,0.009534,0.516020,0.760215,0.161076,0.000000,...,3.0,5.0,1.0,0.111359,0.000000,8.956325e-05,0000006.bin,-1.000000,0.0,6.0


In [88]:
test.head(50)

,DiffcoeffZ12,DiffcoeffZ23,SimID,Stable,Zcosphistd12,Zcosphistd23,Zcross12,Zcross13,Zcross23,Zdetrendedstd12,...,maxi2,maxj,maxk,maxstrength,medvarZ12,medvarZ23,runstring,secondres,secondstrength,seed
0,0.000000e+00,-5.249295e-10,0.0,False,0.000000,0.012035,0.504063,0.594658,0.060234,0.000000,...,3.0,12.0,1.0,0.048244,0.000000,1.471706e-04,0000000.bin,0.195509,0.0,3500000.0
1,5.530677e-11,9.176673e-11,2.0,True,0.003070,0.000785,0.129660,1.261558,1.001981,0.003071,...,2.0,6.0,1.0,0.067762,0.000009,3.508728e-07,0000002.bin,0.029946,0.0,2.0
2,-1.838390e-08,-5.059799e-09,4.0,False,0.023602,0.033524,0.059897,0.332922,0.257596,0.022934,...,2.0,12.0,1.0,0.030171,0.000586,1.147590e-03,0000004.bin,0.140455,0.0,700004.0
3,2.151018e-08,-1.581556e-07,5.0,False,0.011011,0.053589,0.185250,0.259467,0.062617,0.001381,...,3.0,23.0,2.0,0.001197,0.000029,2.971640e-03,0000005.bin,0.567226,0.0,1400005.0
4,0.000000e+00,-5.992155e-10,6.0,False,0.000000,0.009534,0.516020,0.760215,0.161076,0.000000,...,3.0,5.0,1.0,0.111359,0.000000,8.956325e-05,0000006.bin,-1.000000,0.0,6.0
5,5.718216e-06,-3.337570e-07,7.0,False,0.438043,0.177081,0.069323,0.215457,0.136660,0.058753,...,3.0,4.0,1.0,0.003488,0.032111,8.217204e-03,0000007.bin,0.393146,0.0,2100007.0
6,0.000000e+00,-4.990015e-10,8.0,False,0.000000,0.024843,0.492813,0.657701,0.110455,0.000000,...,3.0,7.0,1.0,0.037960,0.000000,6.195017e-04,0000008.bin,-1.000000,0.0,8.0
7,7.087217e-09,-2.932458e-08,10.0,False,0.018350,0.033251,0.130162,0.381586,0.222467,0.013577,...,2.0,6.0,1.0,0.023995,0.000261,1.032598e-03,0000010.bin,0.205764,0.0,700010.0
8,1.371793e-08,3.786168e-09,11.0,False,0.079462,0.084492,0.093278,0.324455,0.211454,0.007644,...,3.0,4.0,1.0,0.556784,0.000062,2.711810e-04,0000011.bin,0.083947,0.0,2100011.0
9,9.646483e-10,4.714798e-09,12.0,True,0.028829,0.015754,0.136068,0.340661,0.180089,0.007188,...,2.0,6.0,1.0,0.001430,0.000051,9.376263e-05,0000012.bin,0.553938,0.0,12.0


In [86]:
test.shape